In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/df.csv')
df.head()

,Round,Game,Team_1,Team_2,Outcome,Probability
0,1,1,A,B,1,0.8
1,1,2,C,D,1,0.8
2,1,3,E,F,1,0.8
3,1,4,G,H,1,0.8
4,1,1,A,B,0,0.2


In [3]:
df['Round_Game'] =  [str(a)+'_'+str(b) for a,b in zip(df['Round'], df['Game'])]

In [4]:
df['Winning_Team'] = [a if o==1 else b for a,b,o in zip(df['Team_1'], df['Team_2'], df['Outcome'])]

In [5]:
for c in set(df['Round_Game']):
    df.loc[df['Round_Game']==c, 'Round_Game_'+c] = 1
    df['Round_Game_'+c].fillna(0, inplace=True)

In [6]:
for r in set(df['Round']):
    for t in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']:
        df[t+'_'+str(r)] = [0 if t!=tm else(a if a==r else -1 if a<r else 0) for a,tm in zip(df['Round'], df['Winning_Team'])]
df        

,Round,Game,Team_1,Team_2,Outcome,Probability,Round_Game,Winning_Team,Round_Game_1_1,Round_Game_2_2,...,G_2,H_2,A_3,B_3,C_3,D_3,E_3,F_3,G_3,H_3
0,1,1,A,B,1,0.8,1_1,A,1.0,0.0,...,0,0,-1,0,0,0,0,0,0,0
1,1,2,C,D,1,0.8,1_2,C,0.0,0.0,...,0,0,0,0,-1,0,0,0,0,0
2,1,3,E,F,1,0.8,1_3,E,0.0,0.0,...,0,0,0,0,0,0,-1,0,0,0
3,1,4,G,H,1,0.8,1_4,G,0.0,0.0,...,-1,0,0,0,0,0,0,0,-1,0
4,1,1,A,B,0,0.2,1_1,B,1.0,0.0,...,0,0,0,-1,0,0,0,0,0,0
5,1,2,C,D,0,0.2,1_2,D,0.0,0.0,...,0,0,0,0,0,-1,0,0,0,0
6,1,3,E,F,0,0.2,1_3,F,0.0,0.0,...,0,0,0,0,0,0,0,-1,0,0
7,1,4,G,H,0,0.2,1_4,H,0.0,0.0,...,0,-1,0,0,0,0,0,0,0,-1
8,2,1,A,C,1,0.8,2_1,A,0.0,0.0,...,0,0,-1,0,0,0,0,0,0,0
9,2,1,A,D,1,0.8,2_1,A,0.0,0.0,...,0,0,-1,0,0,0,0,0,0,0


In [7]:
df.columns.tolist()

['Round',
 'Game',
 'Team_1',
 'Team_2',
 'Outcome',
 'Probability',
 'Round_Game',
 'Winning_Team',
 'Round_Game_1_1',
 'Round_Game_2_2',
 'Round_Game_3_1',
 'Round_Game_1_3',
 'Round_Game_2_1',
 'Round_Game_1_4',
 'Round_Game_1_2',
 'A_1',
 'B_1',
 'C_1',
 'D_1',
 'E_1',
 'F_1',
 'G_1',
 'H_1',
 'A_2',
 'B_2',
 'C_2',
 'D_2',
 'E_2',
 'F_2',
 'G_2',
 'H_2',
 'A_3',
 'B_3',
 'C_3',
 'D_3',
 'E_3',
 'F_3',
 'G_3',
 'H_3']

In [8]:
from ortools.linear_solver import pywraplp
opt_vars = 'Probability'

In [9]:
solver = pywraplp.Solver('SolveIntegerProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
x_list = {}
for i in range(len(df)):
    x_list[i] = solver.IntVar(0.0, 1.0, 'x'+str(i))   

In [10]:
# sets constraints

# add game constraints
for rg in set(df['Round_Game']):
    constraint = solver.Constraint(1, 1)
    for i in range(len(df)):
        constraint.SetCoefficient(x_list[i], int(df.loc[i, 'Round_Game_'+rg]))

# adds constraint for each team/round
for r in set(df['Round']):
    for t in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']:
        col = str(t)+'_'+str(r)
        constraint = solver.Constraint(-999, 1)
        for i in range(len(df)):
           constraint.SetCoefficient(x_list[i], int(df.loc[i, col]))

In [11]:
 # sets objective
obj = solver.Objective()
for i in range(len(df)):
    obj.SetCoefficient(x_list[i], int(df.loc[i, opt_vars]))
obj.SetMaximization()

In [12]:
result_status = solver.Solve()

In [13]:
result_status

0

In [14]:
opt_results = df.copy()

for i in range(len(df)):
    opt_results.loc[i, 'Play'] = x_list[i].solution_value()
    
opt_results = opt_results.loc[opt_results['Play']==1][['Round', 'Game', 'Team_1', 'Team_2', 'Outcome']]


In [15]:
opt_results

,Round,Game,Team_1,Team_2,Outcome
0,1,1,A,B,1
2,1,3,E,F,1
3,1,4,G,H,1
5,1,2,C,D,0
13,2,2,E,H,1
19,2,1,B,D,0
36,3,1,D,E,1


In [19]:
df.to_csv('../data/df_modified.csv', index=False)

In [17]:
df['Round_conversion'] = [10**(x-1) for x in df['Round']]
for team in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']:
    df['Team_'+team] = (df['Team_1']==team)*df['Round_conversion']*df['Outcome'] + (df['Team_2']==team)*df['Round_conversion']*(df['Outcome']==0)
df.head()

,Round,Game,Team_1,Team_2,Outcome,Probability,Round_Game,Winning_Team,Round_Game_1_1,Round_Game_2_2,...,H_3,Round_conversion,Team_A,Team_B,Team_C,Team_D,Team_E,Team_F,Team_G,Team_H
0,1,1,A,B,1,0.8,1_1,A,1.0,0.0,...,0,1,1,0,0,0,0,0,0,0
1,1,2,C,D,1,0.8,1_2,C,0.0,0.0,...,0,1,0,0,1,0,0,0,0,0
2,1,3,E,F,1,0.8,1_3,E,0.0,0.0,...,0,1,0,0,0,0,1,0,0,0
3,1,4,G,H,1,0.8,1_4,G,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0
4,1,1,A,B,0,0.2,1_1,B,1.0,0.0,...,0,1,0,1,0,0,0,0,0,0


In [18]:
df

,Round,Game,Team_1,Team_2,Outcome,Probability,Round_Game,Winning_Team,Round_Game_1_1,Round_Game_2_2,...,H_3,Round_conversion,Team_A,Team_B,Team_C,Team_D,Team_E,Team_F,Team_G,Team_H
0,1,1,A,B,1,0.8,1_1,A,1.0,0.0,...,0,1,1,0,0,0,0,0,0,0
1,1,2,C,D,1,0.8,1_2,C,0.0,0.0,...,0,1,0,0,1,0,0,0,0,0
2,1,3,E,F,1,0.8,1_3,E,0.0,0.0,...,0,1,0,0,0,0,1,0,0,0
3,1,4,G,H,1,0.8,1_4,G,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0
4,1,1,A,B,0,0.2,1_1,B,1.0,0.0,...,0,1,0,1,0,0,0,0,0,0
5,1,2,C,D,0,0.2,1_2,D,0.0,0.0,...,0,1,0,0,0,1,0,0,0,0
6,1,3,E,F,0,0.2,1_3,F,0.0,0.0,...,0,1,0,0,0,0,0,1,0,0
7,1,4,G,H,0,0.2,1_4,H,0.0,0.0,...,-1,1,0,0,0,0,0,0,0,1
8,2,1,A,C,1,0.8,2_1,A,0.0,0.0,...,0,10,10,0,0,0,0,0,0,0
9,2,1,A,D,1,0.8,2_1,A,0.0,0.0,...,0,10,10,0,0,0,0,0,0,0
